# NEUS 642 - Week 2 Homework

We'll continue working with the spike data from class. So first, let's initialize this instance of python and load the datasets, same as in the lecture notebook.

In [ ]:
import numpy as np

In [ ]:
spikes1 = np.loadtxt('data/CRD016c-40-1.csv', delimiter=',')
cell1_label = 'CRD016c-40-1'
spikes2 = np.loadtxt('data/CRD016c-50-1.csv', delimiter=',')
cell2_label = 'CRD016c-50-1'

stim = np.loadtxt('data/stim.csv', delimiter=',')
stim = np.round(stim, 3)
print("spikes1 - number of spike events:", spikes1.shape)
print("spikes2 - number of spike events:", spikes2.shape)
print("stim    - number of stimuli X [freq, start, stop]", stim.shape)

# Tuning curve with error bars

The frequency tuning curve shows the mean spike rate response to each distinct stimulus, but it doesn't indicate which of those responses are signficantly different from the spontaneous spike rate.  Ie, stimuli at which frequencies evoke a reliable response?

One quick way to assess significance is to measure the standard error on the mean (SEM), which is the standard deviation across a list of samples (`x`), divided by the square root of the number of samples:

$SEM = \frac{1}{\sqrt{n}} \sqrt{(\sum_i^n{x(i)^2} / m)}$

Using `numpy`, we can compute SEM with the `np.std` function.

In [ ]:
x = np.array([4,4,4,5,5,5,6,6,6])
n=len(x)
sem = np.std(x)/np.sqrt(n)
print(sem)

## Question 1 - Function to compute spontaneous rate.

It's considered good practice to break down complex analysis into smaller chunks, making it easier to debug and reuse code. To focus on one particular aspect of organization, it's helpful to separate out code that performs computations and code that plots the results.

So let's start with something simple and bite-sized. To assess significance, we need to measure the change in spike rate from the spontaneous level. Let's write another function that calculates spont rate from our data. This should be easy, simply "packaging" code from the lecture inside a function. **BUT** note that the variable containing the spike data is called `spikes` (not `spikes1` or `spikes2`).  Why is that?

Syntax:
```
spont_rate = calc_spont_rate(stim, spikes)
```

In [ ]:
def calc_spont_rate(stim, spikes):
    # Your answer here

    return spont_rate

Test it. This should produce the same answer as we got during lecture.

In [ ]:
calc_spont_rate(stim, spikes1)

And we can call the same function for the second neuron:

In [ ]:
calc_spont_rate(stim, spikes2)

## Question 2 - Function to compute mean and SEM response for each tone

Next step: write a function that takes our `spikes` and `stim` arrays, and returns three arrays: a list of unique frequencies, the mean response to each frequency and the SEM for each frequency. You should be able to borrow a most of the code from the lecture notebook.

Syntax:
```
f, m, sem = tuning_mean_sem(stim, spikes)

```

In [ ]:
def tuning_mean_sem(stim, spikes):
    # Your answer here

    return f, m, sem

Test it with data from the first unit (`spikes1`):

In [ ]:
f, m, sem = tuning_mean_sem(stim, spikes1)

i = 5
print(f"For {f[i]} Hz stimulus, mean response is {m[i]:.3f} spikes/sec, and SEM is {sem[i]:.3f} spikes/sec")

## Question 3 - Make a pretty plot with error bars

Finally: using the results generated by your two functions, write code that generates a tuning curve with error bars to indicate stimuli that produce a significant response. Simple rule of thumb: The measurement of a mean value is signficant with p<0.05 if it is at least **2 SEM** from baseline. The error bars in your plot should indicate 2 SEMs. Points where the error bars do not cross the spont line are signficant responses.

The function `plt.errorbar` should come in handy. Important syntax detail: `errorbar` requires three inputs. `errorbar(x,y,e)`, where x indicates values on the x axis, in this case, frequency.

Don't forget to label your x and y axes with appropriate units.

In [ ]:
import matplotlib.pyplot as plt

spont = calc_spont_rate(stim, spikes1)
f, m, sem = tuning_mean_sem(stim, spikes1)

# Your answer here

In [ ]:
def tuning_mean_sem(stim, spikes):
    m = np.zeros(f_list.shape)
    sem = np.zeros(f_list.shape)
    for j, f in enumerate(f_list):   # for loop to iterate through each unqiue
        b = (stim[:,0]==f)
        start_times = stim[b,1]
        stop_times = stim[b,2]
        n=len(start_times)
        trial_count = len(start_times)
        single_trial_rates = np.zeros(trial_count)
        for i, (s,e) in enumerate(zip(start_times,stop_times)):
            # calculate mean spike rate on each trial
            single_trial_rates[i] = np.sum((spikes>s) & (spikes<=e)) / stim_duration
    
        m[j] = np.mean(single_trial_rates)
        sem[j] = np.std(single_trial_rates)/np.sqrt(n)
    
    return f_list, m, sem

def calc_spont_rate(stim, spikes):
    all_start_times = stim[:,1]
    pre_duration = 0.1
    pre_start_times = all_start_times - pre_duration
    total_trials = len(all_start_times)
    
    spont_single_rates = np.zeros(total_trials)
    for i,(s,e) in enumerate(zip(pre_start_times,all_start_times)):
        spont_single_rates[i] = np.sum((spikes>s) & (spikes<=e))/pre_duration
    spont_rate = np.mean(spont_single_rates)

    return spont_rate

In [ ]:
spont = calc_spont_rate(stim, spikes1)
f, m, sem = tuning_mean_sem(stim, spikes1)

import matplotlib.pyplot as plt

plt.errorbar(f,m,sem*2)
plt.axhline(spont,ls='--')